In [2]:
import os
import time
import pandas as pd
import numpy as np
import warnings
import statsmodels.api as sm
from tqdm import tqdm
import seaborn as sns
import datetime
import time
import matplotlib.pyplot as plt
import statsmodels.stats as stat
import sys
import talib
import scipy.stats as stat
import sympy as sy
from sympy.stats import Normal, cdf
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold,KFold
import sklearn
import joblib
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#import pickle5 as pickle
torch.cuda.empty_cache()
pd.set_option('display.max_columns',None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
tqdm.pandas(desc="my bars:")
warnings.filterwarnings("ignore")

In [5]:
import pickle
work_path = os.path.dirname(os.getcwd())
work_path = os.path.dirname(work_path)
etf = pd.read_pickle(work_path+'\\03_data\\300etf.pkl')
etf = pd.DataFrame(etf)
with open(work_path+'\\03_data\\300etf.pkl', 'wb') as f:
    pickle.dump(etf, f)


,time,code,open,high,low,close,volume
0,2022-05-25 09:35:00,510300.OF,3.9600,3.9760,3.9590,3.9730,31752500
1,2022-05-25 09:40:00,510300.OF,3.9740,3.9740,3.9650,3.9680,19633800
2,2022-05-25 09:45:00,510300.OF,3.9680,3.9680,3.9610,3.9670,31024100
3,2022-05-25 09:50:00,510300.OF,3.9670,3.9750,3.9620,3.9740,21453401
4,2022-05-25 09:55:00,510300.OF,3.9730,3.9810,3.9670,3.9770,9367100
...,...,...,...,...,...,...,...
2155,2022-07-27 14:40:00,510300.OF,4.2720,4.2750,4.2690,4.2690,8770300
2156,2022-07-27 14:45:00,510300.OF,4.2690,4.2740,4.2680,4.2710,9720000
2157,2022-07-27 14:50:00,510300.OF,4.2710,4.2740,4.2700,4.2730,7215500
2158,2022-07-27 14:55:00,510300.OF,4.2730,4.2760,4.2720,4.2760,6578100
